In [1]:
import copy
import numpy as np
import pandas as pd
from fixedincomelib.data import DataCollection, Data1D, Data2D,build_yc_data_collection
from fixedincomelib.date import Date
from fixedincomelib.yield_curve import YieldCurve
from fixedincomelib.valuation import IndexManager
from fixedincomelib.valuation import ValuationEngineRegistry
from fixedincomelib.product import (ProductOvernightCapFloorlet, ProductOvernightSwaption, ProductOvernightCapFloor)
import QuantLib as ql
from fixedincomelib.builders import create_products_from_data1d
from fixedincomelib.builders import build_yc_calibration_basket
from fixedincomelib.product.product_display_visitor import RfrFutureVisitor, OvernightSwapVisitor
from fixedincomelib.utilities.risk_reporting import createValueReport, risk_vectors_to_df
from fixedincomelib.sabr import valuation_engine_sabr, SabrModel
from fixedincomelib.analytics import SABRCalculator


In [2]:
IndexManager.instance()

In [3]:
MARKET_DF = pd.DataFrame(
    [
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-09-24 x 2025-12-24", 95.70],
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-12-24 x 2026-03-24", 95.80],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-03-24 x 2026-06-24", 95.90],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-06-24 x 2026-09-24", 96.00],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-09-24 x 2026-12-24", 96.08],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-12-24 x 2027-03-24", 96.16],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-03-24 x 2027-06-24", 96.24],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-06-24 x 2027-09-24", 96.32],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-09-24 x 2027-12-24", 96.38],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-12-24 x 2028-03-24", 96.44], 
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-03-24 x 2028-06-24", 96.50],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-06-24 x 2028-09-24", 96.55],  
        ["RFR SWAP","USD-SOFR-OIS","4Y",  0.0368],
        ["RFR SWAP","USD-SOFR-OIS","5Y",  0.0365],
        ["RFR SWAP","USD-SOFR-OIS","6Y",  0.0371],
        ["RFR SWAP","USD-SOFR-OIS","7Y",  0.0374],
        ["RFR SWAP","USD-SOFR-OIS","8Y",  0.0380],
        ["RFR SWAP","USD-SOFR-OIS","9Y",  0.0383],
        ["RFR SWAP","USD-SOFR-OIS","10Y", 0.0386],  
        ["RFR SWAP","USD-SOFR-OIS","15Y", 0.0395],
        ["RFR SWAP","USD-SOFR-OIS","20Y", 0.0405],
        ["RFR SWAP","USD-SOFR-OIS","25Y", 0.0412],
        ["RFR SWAP","USD-SOFR-OIS","30Y", 0.0419],  
        ["RFR SWAP","USD-SOFR-OIS","40Y", 0.0423],
        ["RFR SWAP","USD-SOFR-OIS","50Y", 0.0428],
        ["RFR SWAP","USD-SOFR-OIS","60Y", 0.0432],
    ],
    columns=["DATA TYPE","DATA CONVENTION","AXIS","VALUE"],
)


In [4]:
data_objs, dc = build_yc_data_collection(MARKET_DF)

In [5]:
build_methods = [{
    "TARGET": "SOFR-1B",
    "REFERENCE": None,
    "INSTRUMENTS": ["SOFR-FUTURE-3M", "USD-SOFR-OIS"],
    "INTERPOLATION METHOD": "PIECEWISE_CONSTANT",
}]

In [6]:
yc = YieldCurve("2025-09-24", dc, build_methods)
print("Curve components:", yc.components.keys())

Curve components: dict_keys(['SOFR-1B'])


In [7]:
# Grids
caplet_expiries = [1/12, 0.25, 0.5, 1.0, 2.0, 5.0]        
caplet_tenors   = [1/12, 0.25, 0.5]                         

swaption_expiries = [0.25, 0.5, 1.0, 2.0, 5.0, 10.0]        
swaption_tenors   = [1.0, 2.0, 5.0, 10.0, 30.0]             
INDEX = "SOFR-1B"
beta_fixed = 0.6


def caplet_atm_normal(expiry, tenor):
    base = 0.0100 * np.exp(-0.35 * expiry) + 0.0032
    tenor_adj = {1/12: 1.05, 0.25: 1.00, 0.5: 0.96}[tenor]
    return base * tenor_adj

def swaption_atm_normal(expiry, tenor):
    base = 0.0085 * np.exp(-0.25 * expiry) + 0.0022
    tenor_adj = {1.0: 1.05, 2.0: 1.00, 5.0: 0.92, 10.0: 0.86, 30.0: 0.80}[tenor]
    return base * tenor_adj

def nu_surface(expiry, tenor, product):
    if product == "CAPLET":
        return (1.10 * np.exp(-0.40 * expiry) + 0.18) * (0.95 + 0.10*np.exp(-2.0*tenor))
    else:  
        return (0.85 * np.exp(-0.30 * expiry) + 0.12) * (1.00 - 0.05*np.log(1.0 + tenor))

def rho_surface(expiry, tenor, product):
    if product == "CAPLET":
        base = -0.32 + 0.06*np.log(1.0 + expiry)
        tenor_adj = 0.04*np.log(1.0 + 12*tenor)   # tenor in years -> scale up
        return np.clip(base + tenor_adj, -0.55, -0.05)
    else: 
        base = -0.38 + 0.07*np.log(1.0 + expiry)
        tenor_adj = 0.05*np.log(1.0 + tenor)
        return np.clip(base + tenor_adj, -0.60, -0.05)

rows = []

# CAPLETS
for e in caplet_expiries:
    for t in caplet_tenors:
        rows.append([INDEX, float(e), float(t),
                     float(caplet_atm_normal(e,t)),
                     float(beta_fixed),
                     float(nu_surface(e,t,"CAPLET")),
                     float(rho_surface(e,t,"CAPLET")),
                     "CAPLET"])

# SWAPTIONS
for e in swaption_expiries:
    for t in swaption_tenors:
        rows.append([INDEX, float(e), float(t),
                     float(swaption_atm_normal(e,t)),
                     float(beta_fixed),
                     float(nu_surface(e,t,"SWAPTION")),
                     float(rho_surface(e,t,"SWAPTION")),
                     "SWAPTION"])

sabr_sofr_topdown = pd.DataFrame(rows, columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO","PRODUCT"])

sabr_sofr_topdown


,INDEX,AXIS1,AXIS2,NORMALVOL,BETA,NU,RHO,PRODUCT
0,SOFR-1B,0.083333,0.083333,0.013558,0.600000,1.287038,-0.287472,CAPLET
1,SOFR-1B,0.083333,0.250000,0.012913,0.600000,1.257189,-0.259746,CAPLET
2,SOFR-1B,0.083333,0.500000,0.012396,0.600000,1.227503,-0.237361,CAPLET
3,SOFR-1B,0.250000,0.083333,0.012980,0.600000,1.216044,-0.278885,CAPLET
4,SOFR-1B,0.250000,0.250000,0.012362,0.600000,1.187842,-0.251160,CAPLET
5,SOFR-1B,0.250000,0.500000,0.011868,0.600000,1.159793,-0.228775,CAPLET
6,SOFR-1B,0.500000,0.083333,0.012174,0.600000,1.118045,-0.267946,CAPLET
7,SOFR-1B,0.500000,0.250000,0.011595,0.600000,1.092116,-0.240220,CAPLET
8,SOFR-1B,0.500000,0.500000,0.011131,0.600000,1.066327,-0.217836,CAPLET
9,SOFR-1B,1.000000,0.083333,0.010759,0.600000,0.949137,-0.250685,CAPLET


In [8]:
data_objs = []
for idx_name, sub in sabr_sofr_topdown.groupby("INDEX"):
    for param in ["NORMALVOL","BETA","NU","RHO"]:
        pivot = (
            sub.pivot(index="AXIS1", columns="AXIS2", values=param)
               .sort_index(axis=0).sort_index(axis=1)
        )
        data_objs.append(Data2D.createDataObject(param.lower(), idx_name, pivot))


In [9]:
value_date = "2025-09-24"

topdown_build_methods = []
for product in ("CAPLET","SWAPTION"):
    for param in ("NORMALVOL","BETA","NU","RHO"):
        topdown_build_methods.append({
            "TARGET":           "SOFR-1B",
            "VALUES":           param,
            "AXIS1":            "AXIS1",
            "AXIS2":            "AXIS2",
            "INTERPOLATION":    "LINEAR",
            "SHIFT":            0.0,
            "VOL_DECAY_SPEED":  0.2,
            "PRODUCT":          product
        })

dc = DataCollection(data_objs)

td_sabr = SabrModel.from_curve(
    valueDate=value_date,
    dataCollection=dc,
    buildMethodCollection=topdown_build_methods,
    ycModel=yc
)

print("Top-down SABR components:", td_sabr.components.keys())

Top-down SABR components: dict_keys(['SOFR-1B-NORMALVOL-CAPLET', 'SOFR-1B-BETA-CAPLET', 'SOFR-1B-NU-CAPLET', 'SOFR-1B-RHO-CAPLET', 'SOFR-1B-NORMALVOL-SWAPTION', 'SOFR-1B-BETA-SWAPTION', 'SOFR-1B-NU-SWAPTION', 'SOFR-1B-RHO-SWAPTION'])


In [10]:
np.set_printoptions(threshold=np.inf, linewidth=200, suppress=True)
td_sabr.jacobian()

array([[-100.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.  

In [11]:
td_cap = ProductOvernightCapFloorlet(
    effectiveDate="2026-07-01",
    termOrEnd    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="CAP",
    strike       =0.018,
    notional     =1000000,
    longOrShort  ="LONG"
)

vp = {"SABR_METHOD": "top-down", "FUNDING INDEX": "SOFR-1B"}  # method will be forced to Hagan anyway
ve = ValuationEngineRegistry().new_valuation_engine(td_sabr, vp, td_cap)

ve.calculateValue()
print("Cap PV:", ve.value)

ve.calculateFirstOrderRisk()
g = np.asarray(ve.firstOrderRisk, dtype=float)
print(g)

a= ve.value[1]


Cap PV: ['USD', np.float64(5467.253520623842)]
[ -1382.00019549  -1366.81338016  -1397.18701083 219656.17236108  18098.08659387      0.              0.              0.              0.              0.              0.              0.
      0.              0.              0.              0.              0.              0.              0.              0.              0.              0.              0.              0.
      0.              0.              0.              0.              0.              0.              0.              0.              0.              0.              0.              0.
      0.              0.              0.              0.              0.              0.              0.           9292.45391785    211.1921345       0.              0.              0.
      0.              0.              0.          11615.56739732    263.99016812      0.              0.              0.              0.              0.              0.              0.
      0.              0.    

In [12]:
report = createValueReport(vp, td_sabr, td_cap, request="all", space="pv")
g = report["param_risk"]
print(g.shape, g)

(474,) [   13.82000195    13.81669882    14.27203581 -2267.26125099  -188.71557022    -0.            -0.            -0.            -0.            -0.            -0.            -0.            -0.
    -0.            -0.            -0.            -0.            -0.            -0.            -0.            -0.            -0.            -0.            -0.            -0.            -0.
     0.             0.             0.             0.             0.             0.             0.             0.             0.             0.             0.             0.             0.
     0.             0.             0.             0.          9292.45391785   211.1921345      0.             0.             0.             0.             0.             0.         11615.56739732
   263.99016812     0.             0.             0.             0.             0.             0.             0.             0.             0.             0.             0.             0.
     0.             0.             0.        

In [13]:
report_q = createValueReport(vp, td_sabr, td_cap, request="all", space="quote")
dq = report_q["quote_risk"]
print(dq.shape)
print(dq)

(474,)
[  -13.67140142   -13.52610965   -13.82699394  2174.33518615   179.2051421      0.             0.             0.             0.             0.             0.             0.            -0.
    -0.            -0.            -0.            -0.            -0.            -0.            -0.            -0.            -0.            -0.            -0.            -0.            -0.
     0.             0.             0.             0.             0.             0.             0.             0.             0.             0.             0.             0.             0.
     0.             0.             0.             0.          9292.45391785   211.1921345      0.             0.             0.             0.             0.             0.         11615.56739732
   263.99016812     0.             0.             0.             0.             0.             0.             0.             0.             0.             0.             0.             0.
     0.             0.             0.        

In [14]:
pd.set_option("display.float_format", lambda x: f"{x:,.6f}")

In [15]:
df_report = risk_vectors_to_df(td_sabr,report_q, yc_index="SOFR-1B") 

df_report.head(5)

,block,index,param,expiry,tenor,node,pos,dPV_dModelParam,hedgeWeightPV,dPV_dQuote,abs_model,abs_weight,abs_quote
0,YC,SOFR-1B,IFR,NaN,NaN,"RFR FUTURE SOFR-FUTURE-3M September 24th, 2025...",0,"-1,382.000195",13.820002,-13.671401,"1,382.000195",13.820002,13.671401
1,YC,SOFR-1B,IFR,NaN,NaN,"RFR FUTURE SOFR-FUTURE-3M December 24th, 2025 ...",1,"-1,366.813380",13.816699,-13.526110,"1,366.813380",13.816699,13.526110
2,YC,SOFR-1B,IFR,NaN,NaN,"RFR FUTURE SOFR-FUTURE-3M March 24th, 2026 x J...",2,"-1,397.187011",14.272036,-13.826994,"1,397.187011",14.272036,13.826994
3,YC,SOFR-1B,IFR,NaN,NaN,"RFR FUTURE SOFR-FUTURE-3M June 24th, 2026 x Se...",3,"219,656.172361","-2,267.261251","2,174.335186","219,656.172361","2,267.261251","2,174.335186"
4,YC,SOFR-1B,IFR,NaN,NaN,"RFR FUTURE SOFR-FUTURE-3M September 24th, 2026...",4,"18,098.086594",-188.715570,179.205142,"18,098.086594",188.715570,179.205142


In [16]:
MARKET_DF2 = pd.DataFrame(
    [
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-09-24 x 2025-12-24", 95.71],
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-12-24 x 2026-03-24", 95.80],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-03-24 x 2026-06-24", 95.90],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-06-24 x 2026-09-24", 96.00],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-09-24 x 2026-12-24", 96.08],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-12-24 x 2027-03-24", 96.16],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-03-24 x 2027-06-24", 96.24],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-06-24 x 2027-09-24", 96.32],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-09-24 x 2027-12-24", 96.38],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-12-24 x 2028-03-24", 96.44], 
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-03-24 x 2028-06-24", 96.50],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-06-24 x 2028-09-24", 96.55],  
        ["RFR SWAP","USD-SOFR-OIS","4Y",  0.0368],
        ["RFR SWAP","USD-SOFR-OIS","5Y",  0.0365],
        ["RFR SWAP","USD-SOFR-OIS","6Y",  0.0371],
        ["RFR SWAP","USD-SOFR-OIS","7Y",  0.0374],
        ["RFR SWAP","USD-SOFR-OIS","8Y",  0.0380],
        ["RFR SWAP","USD-SOFR-OIS","9Y",  0.0383],
        ["RFR SWAP","USD-SOFR-OIS","10Y", 0.0386],  
        ["RFR SWAP","USD-SOFR-OIS","15Y", 0.0395],
        ["RFR SWAP","USD-SOFR-OIS","20Y", 0.0405],
        ["RFR SWAP","USD-SOFR-OIS","25Y", 0.0412],
        ["RFR SWAP","USD-SOFR-OIS","30Y", 0.0419],  
        ["RFR SWAP","USD-SOFR-OIS","40Y", 0.0423],
        ["RFR SWAP","USD-SOFR-OIS","50Y", 0.0428],
        ["RFR SWAP","USD-SOFR-OIS","60Y", 0.0432],
    ],
    columns=["DATA TYPE","DATA CONVENTION","AXIS","VALUE"],
)


In [17]:
data_objs2, dc2 = build_yc_data_collection(MARKET_DF2)

In [18]:
build_methods = [{
    "TARGET": "SOFR-1B",
    "REFERENCE": None,
    "INSTRUMENTS": ["SOFR-FUTURE-3M", "USD-SOFR-OIS"],
    "INTERPOLATION METHOD": "PIECEWISE_CONSTANT",
}]



In [19]:
yc2 = YieldCurve("2025-09-24", dc2, build_methods)
print("Curve components:", yc2.components.keys())

Curve components: dict_keys(['SOFR-1B'])


In [20]:
value_date = "2025-09-24"

topdown_build_methods = []
for product in ("CAPLET","SWAPTION"):
    for param in ("NORMALVOL","BETA","NU","RHO"):
        topdown_build_methods.append({
            "TARGET":           "SOFR-1B",
            "VALUES":           param,
            "AXIS1":            "AXIS1",
            "AXIS2":            "AXIS2",
            "INTERPOLATION":    "LINEAR",
            "SHIFT":            0.0,
            "VOL_DECAY_SPEED":  0.2,
            "PRODUCT":          product
        })

dc = DataCollection(data_objs)

td_sabr2 = SabrModel.from_curve(
    valueDate=value_date,
    dataCollection=dc,
    buildMethodCollection=topdown_build_methods,
    ycModel=yc2
)

print(td_sabr2.components.keys())

dict_keys(['SOFR-1B-NORMALVOL-CAPLET', 'SOFR-1B-BETA-CAPLET', 'SOFR-1B-NU-CAPLET', 'SOFR-1B-RHO-CAPLET', 'SOFR-1B-NORMALVOL-SWAPTION', 'SOFR-1B-BETA-SWAPTION', 'SOFR-1B-NU-SWAPTION', 'SOFR-1B-RHO-SWAPTION'])


In [21]:
td_cap = ProductOvernightCapFloorlet(
    effectiveDate="2026-07-01",
    termOrEnd    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="CAP",
    strike       =0.018,
    notional     =1000000,
    longOrShort  ="LONG"
)

vp = {"SABR_METHOD": "top-down", "FUNDING INDEX": "SOFR-1B"}  # method will be forced to Hagan anyway
ve = ValuationEngineRegistry().new_valuation_engine(td_sabr2, vp, td_cap)

ve.calculateValue()

b = ve.value[1]

In [22]:
(b-a)/0.01

np.float64(13.671741095367906)

In [23]:
# Grids
caplet_expiries = [1/12, 0.25, 0.5, 1.0, 2.0, 5.0]        
caplet_tenors   = [1/12, 0.25, 0.5]                         

swaption_expiries = [0.25, 0.5, 1.0, 2.0, 5.0, 10.0]        
swaption_tenors   = [1.0, 2.0, 5.0, 10.0, 30.0]             
INDEX = "SOFR-1B"
beta_fixed = 0.6


def caplet_atm_normal(expiry, tenor):
    base = 0.0100 * np.exp(-0.35 * expiry) + 0.0032
    tenor_adj = {1/12: 1.05, 0.25: 1.00, 0.5: 0.96}[tenor]
    return base * tenor_adj

def swaption_atm_normal(expiry, tenor):
    base = 0.0085 * np.exp(-0.25 * expiry) + 0.0022
    tenor_adj = {1.0: 1.05, 2.0: 1.00, 5.0: 0.92, 10.0: 0.86, 30.0: 0.80}[tenor]
    return base * tenor_adj

def nu_surface(expiry, tenor, product):
    if product == "CAPLET":
        return (1.10 * np.exp(-0.40 * expiry) + 0.18) * (0.95 + 0.10*np.exp(-2.0*tenor))
    else:  
        return (0.85 * np.exp(-0.30 * expiry) + 0.12) * (1.00 - 0.05*np.log(1.0 + tenor))

def rho_surface(expiry, tenor, product):
    if product == "CAPLET":
        base = -0.32 + 0.06*np.log(1.0 + expiry)
        tenor_adj = 0.04*np.log(1.0 + 12*tenor)   # tenor in years -> scale up
        return np.clip(base + tenor_adj, -0.55, -0.05)
    else: 
        base = -0.38 + 0.07*np.log(1.0 + expiry)
        tenor_adj = 0.05*np.log(1.0 + tenor)
        return np.clip(base + tenor_adj, -0.60, -0.05)

rows = []

# CAPLETS
for e in caplet_expiries:
    for t in caplet_tenors:
        rows.append([INDEX, float(e), float(t),
                     float(caplet_atm_normal(e,t)),
                     float(beta_fixed),
                     float(nu_surface(e,t,"CAPLET")),
                     float(rho_surface(e,t,"CAPLET")),
                     "CAPLET"])

# SWAPTIONS
for e in swaption_expiries:
    for t in swaption_tenors:
        rows.append([INDEX, float(e), float(t),
                     float(swaption_atm_normal(e,t)),
                     float(beta_fixed),
                     float(nu_surface(e,t,"SWAPTION")),
                     float(rho_surface(e,t,"SWAPTION")),
                     "SWAPTION"])

sabr_sofr_topdown2 = pd.DataFrame(rows, columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO","PRODUCT"])

m = (
    (sabr_sofr_topdown2["INDEX"] == "SOFR-1B") &
    np.isclose(sabr_sofr_topdown2["AXIS1"], 1.0) &
    np.isclose(sabr_sofr_topdown2["AXIS2"], 0.25)
)

sabr_sofr_topdown2.loc[m, "NORMALVOL"] += 0.0001

sabr_sofr_topdown2


,INDEX,AXIS1,AXIS2,NORMALVOL,BETA,NU,RHO,PRODUCT
0,SOFR-1B,0.083333,0.083333,0.013558,0.600000,1.287038,-0.287472,CAPLET
1,SOFR-1B,0.083333,0.250000,0.012913,0.600000,1.257189,-0.259746,CAPLET
2,SOFR-1B,0.083333,0.500000,0.012396,0.600000,1.227503,-0.237361,CAPLET
3,SOFR-1B,0.250000,0.083333,0.012980,0.600000,1.216044,-0.278885,CAPLET
4,SOFR-1B,0.250000,0.250000,0.012362,0.600000,1.187842,-0.251160,CAPLET
5,SOFR-1B,0.250000,0.500000,0.011868,0.600000,1.159793,-0.228775,CAPLET
6,SOFR-1B,0.500000,0.083333,0.012174,0.600000,1.118045,-0.267946,CAPLET
7,SOFR-1B,0.500000,0.250000,0.011595,0.600000,1.092116,-0.240220,CAPLET
8,SOFR-1B,0.500000,0.500000,0.011131,0.600000,1.066327,-0.217836,CAPLET
9,SOFR-1B,1.000000,0.083333,0.010759,0.600000,0.949137,-0.250685,CAPLET


In [24]:
data_objs2 = []
for idx_name, sub in sabr_sofr_topdown2.groupby("INDEX"):
    for param in ["NORMALVOL","BETA","NU","RHO"]:
        pivot = (
            sub.pivot(index="AXIS1", columns="AXIS2", values=param)
               .sort_index(axis=0).sort_index(axis=1)
        )
        data_objs2.append(Data2D.createDataObject(param.lower(), idx_name, pivot))


In [25]:
value_date = "2025-09-24"

topdown_build_methods = []
for product in ("CAPLET","SWAPTION"):
    for param in ("NORMALVOL","BETA","NU","RHO"):
        topdown_build_methods.append({
            "TARGET":           "SOFR-1B",
            "VALUES":           param,
            "AXIS1":            "AXIS1",
            "AXIS2":            "AXIS2",
            "INTERPOLATION":    "LINEAR",
            "SHIFT":            0.0,
            "VOL_DECAY_SPEED":  0.2,
            "PRODUCT":          product
        })

dc2 = DataCollection(data_objs2)

td_sabr3 = SabrModel.from_curve(
    valueDate=value_date,
    dataCollection=dc2,
    buildMethodCollection=topdown_build_methods,
    ycModel=yc
)

print(td_sabr3.components.keys())

dict_keys(['SOFR-1B-NORMALVOL-CAPLET', 'SOFR-1B-BETA-CAPLET', 'SOFR-1B-NU-CAPLET', 'SOFR-1B-RHO-CAPLET', 'SOFR-1B-NORMALVOL-SWAPTION', 'SOFR-1B-BETA-SWAPTION', 'SOFR-1B-NU-SWAPTION', 'SOFR-1B-RHO-SWAPTION'])


In [26]:
td_cap = ProductOvernightCapFloorlet(
    effectiveDate="2026-07-01",
    termOrEnd    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="CAP",
    strike       =0.018,
    notional     =1000000,
    longOrShort  ="LONG"
)

vp = {"SABR_METHOD": "top-down", "FUNDING INDEX": "SOFR-1B"}  # method will be forced to Hagan anyway
ve = ValuationEngineRegistry().new_valuation_engine(td_sabr3, vp, td_cap)

ve.calculateValue()

c = ve.value[1]

In [27]:
(c-a)/0.0001

np.float64(11650.095700288148)

In [28]:
# Grids
caplet_expiries = [1/12, 0.25, 0.5, 1.0, 2.0, 5.0]        
caplet_tenors   = [1/12, 0.25, 0.5]                         

swaption_expiries = [0.25, 0.5, 1.0, 2.0, 5.0, 10.0]        
swaption_tenors   = [1.0, 2.0, 5.0, 10.0, 30.0]             
INDEX = "SOFR-1B"
beta_fixed = 0.6


def caplet_atm_normal(expiry, tenor):
    base = 0.0100 * np.exp(-0.35 * expiry) + 0.0032
    tenor_adj = {1/12: 1.05, 0.25: 1.00, 0.5: 0.96}[tenor]
    return base * tenor_adj

def swaption_atm_normal(expiry, tenor):
    base = 0.0085 * np.exp(-0.25 * expiry) + 0.0022
    tenor_adj = {1.0: 1.05, 2.0: 1.00, 5.0: 0.92, 10.0: 0.86, 30.0: 0.80}[tenor]
    return base * tenor_adj

def nu_surface(expiry, tenor, product):
    if product == "CAPLET":
        return (1.10 * np.exp(-0.40 * expiry) + 0.18) * (0.95 + 0.10*np.exp(-2.0*tenor))
    else:  
        return (0.85 * np.exp(-0.30 * expiry) + 0.12) * (1.00 - 0.05*np.log(1.0 + tenor))

def rho_surface(expiry, tenor, product):
    if product == "CAPLET":
        base = -0.32 + 0.06*np.log(1.0 + expiry)
        tenor_adj = 0.04*np.log(1.0 + 12*tenor)   # tenor in years -> scale up
        return np.clip(base + tenor_adj, -0.55, -0.05)
    else: 
        base = -0.38 + 0.07*np.log(1.0 + expiry)
        tenor_adj = 0.05*np.log(1.0 + tenor)
        return np.clip(base + tenor_adj, -0.60, -0.05)

rows = []

# CAPLETS
for e in caplet_expiries:
    for t in caplet_tenors:
        rows.append([INDEX, float(e), float(t),
                     float(caplet_atm_normal(e,t)),
                     float(beta_fixed),
                     float(nu_surface(e,t,"CAPLET")),
                     float(rho_surface(e,t,"CAPLET")),
                     "CAPLET"])

# SWAPTIONS
for e in swaption_expiries:
    for t in swaption_tenors:
        rows.append([INDEX, float(e), float(t),
                     float(swaption_atm_normal(e,t)),
                     float(beta_fixed),
                     float(nu_surface(e,t,"SWAPTION")),
                     float(rho_surface(e,t,"SWAPTION")),
                     "SWAPTION"])

sabr_sofr_topdown3 = pd.DataFrame(rows, columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO","PRODUCT"])

m = (
    (sabr_sofr_topdown2["INDEX"] == "SOFR-1B") &
    np.isclose(sabr_sofr_topdown2["AXIS1"], 1.0) &
    np.isclose(sabr_sofr_topdown2["AXIS2"], 0.25)
)

sabr_sofr_topdown3.loc[m, "BETA"] += 0.01

sabr_sofr_topdown3


,INDEX,AXIS1,AXIS2,NORMALVOL,BETA,NU,RHO,PRODUCT
0,SOFR-1B,0.083333,0.083333,0.013558,0.600000,1.287038,-0.287472,CAPLET
1,SOFR-1B,0.083333,0.250000,0.012913,0.600000,1.257189,-0.259746,CAPLET
2,SOFR-1B,0.083333,0.500000,0.012396,0.600000,1.227503,-0.237361,CAPLET
3,SOFR-1B,0.250000,0.083333,0.012980,0.600000,1.216044,-0.278885,CAPLET
4,SOFR-1B,0.250000,0.250000,0.012362,0.600000,1.187842,-0.251160,CAPLET
5,SOFR-1B,0.250000,0.500000,0.011868,0.600000,1.159793,-0.228775,CAPLET
6,SOFR-1B,0.500000,0.083333,0.012174,0.600000,1.118045,-0.267946,CAPLET
7,SOFR-1B,0.500000,0.250000,0.011595,0.600000,1.092116,-0.240220,CAPLET
8,SOFR-1B,0.500000,0.500000,0.011131,0.600000,1.066327,-0.217836,CAPLET
9,SOFR-1B,1.000000,0.083333,0.010759,0.600000,0.949137,-0.250685,CAPLET


In [29]:
data_objs3 = []
for idx_name, sub in sabr_sofr_topdown3.groupby("INDEX"):
    for param in ["NORMALVOL","BETA","NU","RHO"]:
        pivot = (
            sub.pivot(index="AXIS1", columns="AXIS2", values=param)
               .sort_index(axis=0).sort_index(axis=1)
        )
        data_objs3.append(Data2D.createDataObject(param.lower(), idx_name, pivot))


In [30]:
value_date = "2025-09-24"

topdown_build_methods = []
for product in ("CAPLET","SWAPTION"):
    for param in ("NORMALVOL","BETA","NU","RHO"):
        topdown_build_methods.append({
            "TARGET":           "SOFR-1B",
            "VALUES":           param,
            "AXIS1":            "AXIS1",
            "AXIS2":            "AXIS2",
            "INTERPOLATION":    "LINEAR",
            "SHIFT":            0.0,
            "VOL_DECAY_SPEED":  0.2,
            "PRODUCT":          product
        })

dc3 = DataCollection(data_objs3)

td_sabr4 = SabrModel.from_curve(
    valueDate=value_date,
    dataCollection=dc3,
    buildMethodCollection=topdown_build_methods,
    ycModel=yc
)

print(td_sabr4.components.keys())

dict_keys(['SOFR-1B-NORMALVOL-CAPLET', 'SOFR-1B-BETA-CAPLET', 'SOFR-1B-NU-CAPLET', 'SOFR-1B-RHO-CAPLET', 'SOFR-1B-NORMALVOL-SWAPTION', 'SOFR-1B-BETA-SWAPTION', 'SOFR-1B-NU-SWAPTION', 'SOFR-1B-RHO-SWAPTION'])


In [31]:
td_cap = ProductOvernightCapFloorlet(
    effectiveDate="2026-07-01",
    termOrEnd    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="CAP",
    strike       =0.018,
    notional     =1000000,
    longOrShort  ="LONG"
)

vp = {"SABR_METHOD": "top-down", "FUNDING INDEX": "SOFR-1B"}
ve = ValuationEngineRegistry().new_valuation_engine(td_sabr4, vp, td_cap)

ve.calculateValue()

d = ve.value[1]

In [32]:
(d-a)/0.01

np.float64(-46.684980879763316)

In [33]:
# Grids
caplet_expiries = [1/12, 0.25, 0.5, 1.0, 2.0, 5.0]        
caplet_tenors   = [1/12, 0.25, 0.5]                         

swaption_expiries = [0.25, 0.5, 1.0, 2.0, 5.0, 10.0]        
swaption_tenors   = [1.0, 2.0, 5.0, 10.0, 30.0]             
INDEX = "SOFR-1B"
beta_fixed = 0.6


def caplet_atm_normal(expiry, tenor):
    base = 0.0100 * np.exp(-0.35 * expiry) + 0.0032
    tenor_adj = {1/12: 1.05, 0.25: 1.00, 0.5: 0.96}[tenor]
    return base * tenor_adj

def swaption_atm_normal(expiry, tenor):
    base = 0.0085 * np.exp(-0.25 * expiry) + 0.0022
    tenor_adj = {1.0: 1.05, 2.0: 1.00, 5.0: 0.92, 10.0: 0.86, 30.0: 0.80}[tenor]
    return base * tenor_adj

def nu_surface(expiry, tenor, product):
    if product == "CAPLET":
        return (1.10 * np.exp(-0.40 * expiry) + 0.18) * (0.95 + 0.10*np.exp(-2.0*tenor))
    else:  
        return (0.85 * np.exp(-0.30 * expiry) + 0.12) * (1.00 - 0.05*np.log(1.0 + tenor))

def rho_surface(expiry, tenor, product):
    if product == "CAPLET":
        base = -0.32 + 0.06*np.log(1.0 + expiry)
        tenor_adj = 0.04*np.log(1.0 + 12*tenor)   # tenor in years -> scale up
        return np.clip(base + tenor_adj, -0.55, -0.05)
    else: 
        base = -0.38 + 0.07*np.log(1.0 + expiry)
        tenor_adj = 0.05*np.log(1.0 + tenor)
        return np.clip(base + tenor_adj, -0.60, -0.05)

rows = []

# CAPLETS
for e in caplet_expiries:
    for t in caplet_tenors:
        rows.append([INDEX, float(e), float(t),
                     float(caplet_atm_normal(e,t)),
                     float(beta_fixed),
                     float(nu_surface(e,t,"CAPLET")),
                     float(rho_surface(e,t,"CAPLET")),
                     "CAPLET"])

# SWAPTIONS
for e in swaption_expiries:
    for t in swaption_tenors:
        rows.append([INDEX, float(e), float(t),
                     float(swaption_atm_normal(e,t)),
                     float(beta_fixed),
                     float(nu_surface(e,t,"SWAPTION")),
                     float(rho_surface(e,t,"SWAPTION")),
                     "SWAPTION"])

sabr_sofr_topdown4 = pd.DataFrame(rows, columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO","PRODUCT"])

m = (
    (sabr_sofr_topdown2["INDEX"] == "SOFR-1B") &
    np.isclose(sabr_sofr_topdown2["AXIS1"], 1.0) &
    np.isclose(sabr_sofr_topdown2["AXIS2"], 0.25)
)

sabr_sofr_topdown4.loc[m, "NU"] += 0.01

sabr_sofr_topdown4


,INDEX,AXIS1,AXIS2,NORMALVOL,BETA,NU,RHO,PRODUCT
0,SOFR-1B,0.083333,0.083333,0.013558,0.600000,1.287038,-0.287472,CAPLET
1,SOFR-1B,0.083333,0.250000,0.012913,0.600000,1.257189,-0.259746,CAPLET
2,SOFR-1B,0.083333,0.500000,0.012396,0.600000,1.227503,-0.237361,CAPLET
3,SOFR-1B,0.250000,0.083333,0.012980,0.600000,1.216044,-0.278885,CAPLET
4,SOFR-1B,0.250000,0.250000,0.012362,0.600000,1.187842,-0.251160,CAPLET
5,SOFR-1B,0.250000,0.500000,0.011868,0.600000,1.159793,-0.228775,CAPLET
6,SOFR-1B,0.500000,0.083333,0.012174,0.600000,1.118045,-0.267946,CAPLET
7,SOFR-1B,0.500000,0.250000,0.011595,0.600000,1.092116,-0.240220,CAPLET
8,SOFR-1B,0.500000,0.500000,0.011131,0.600000,1.066327,-0.217836,CAPLET
9,SOFR-1B,1.000000,0.083333,0.010759,0.600000,0.949137,-0.250685,CAPLET


In [34]:
data_objs4 = []
for idx_name, sub in sabr_sofr_topdown4.groupby("INDEX"):
    for param in ["NORMALVOL","BETA","NU","RHO"]:
        pivot = (
            sub.pivot(index="AXIS1", columns="AXIS2", values=param)
               .sort_index(axis=0).sort_index(axis=1)
        )
        data_objs4.append(Data2D.createDataObject(param.lower(), idx_name, pivot))

In [35]:
value_date = "2025-09-24"

topdown_build_methods = []
for product in ("CAPLET","SWAPTION"):
    for param in ("NORMALVOL","BETA","NU","RHO"):
        topdown_build_methods.append({
            "TARGET":           "SOFR-1B",
            "VALUES":           param,
            "AXIS1":            "AXIS1",
            "AXIS2":            "AXIS2",
            "INTERPOLATION":    "LINEAR",
            "SHIFT":            0.0,
            "VOL_DECAY_SPEED":  0.2,
            "PRODUCT":          product
        })

dc4 = DataCollection(data_objs4)

td_sabr5 = SabrModel.from_curve(
    valueDate=value_date,
    dataCollection=dc4,
    buildMethodCollection=topdown_build_methods,
    ycModel=yc
)

print(td_sabr5.components.keys())

dict_keys(['SOFR-1B-NORMALVOL-CAPLET', 'SOFR-1B-BETA-CAPLET', 'SOFR-1B-NU-CAPLET', 'SOFR-1B-RHO-CAPLET', 'SOFR-1B-NORMALVOL-SWAPTION', 'SOFR-1B-BETA-SWAPTION', 'SOFR-1B-NU-SWAPTION', 'SOFR-1B-RHO-SWAPTION'])


In [36]:
td_cap = ProductOvernightCapFloorlet(
    effectiveDate="2026-07-01",
    termOrEnd    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="CAP",
    strike       =0.018,
    notional     =1000000,
    longOrShort  ="LONG"
)

vp = {"SABR_METHOD": "top-down", "FUNDING INDEX": "SOFR-1B"}
ve = ValuationEngineRegistry().new_valuation_engine(td_sabr5, vp, td_cap)

ve.calculateValue()

e = ve.value[1]

In [37]:
(e-a)/0.01

np.float64(114.50032547918454)

In [38]:
# Grids
caplet_expiries = [1/12, 0.25, 0.5, 1.0, 2.0, 5.0]        
caplet_tenors   = [1/12, 0.25, 0.5]                         

swaption_expiries = [0.25, 0.5, 1.0, 2.0, 5.0, 10.0]        
swaption_tenors   = [1.0, 2.0, 5.0, 10.0, 30.0]             
INDEX = "SOFR-1B"
beta_fixed = 0.6


def caplet_atm_normal(expiry, tenor):
    base = 0.0100 * np.exp(-0.35 * expiry) + 0.0032
    tenor_adj = {1/12: 1.05, 0.25: 1.00, 0.5: 0.96}[tenor]
    return base * tenor_adj

def swaption_atm_normal(expiry, tenor):
    base = 0.0085 * np.exp(-0.25 * expiry) + 0.0022
    tenor_adj = {1.0: 1.05, 2.0: 1.00, 5.0: 0.92, 10.0: 0.86, 30.0: 0.80}[tenor]
    return base * tenor_adj

def nu_surface(expiry, tenor, product):
    if product == "CAPLET":
        return (1.10 * np.exp(-0.40 * expiry) + 0.18) * (0.95 + 0.10*np.exp(-2.0*tenor))
    else:  
        return (0.85 * np.exp(-0.30 * expiry) + 0.12) * (1.00 - 0.05*np.log(1.0 + tenor))

def rho_surface(expiry, tenor, product):
    if product == "CAPLET":
        base = -0.32 + 0.06*np.log(1.0 + expiry)
        tenor_adj = 0.04*np.log(1.0 + 12*tenor)   # tenor in years -> scale up
        return np.clip(base + tenor_adj, -0.55, -0.05)
    else: 
        base = -0.38 + 0.07*np.log(1.0 + expiry)
        tenor_adj = 0.05*np.log(1.0 + tenor)
        return np.clip(base + tenor_adj, -0.60, -0.05)

rows = []

# CAPLETS
for e in caplet_expiries:
    for t in caplet_tenors:
        rows.append([INDEX, float(e), float(t),
                     float(caplet_atm_normal(e,t)),
                     float(beta_fixed),
                     float(nu_surface(e,t,"CAPLET")),
                     float(rho_surface(e,t,"CAPLET")),
                     "CAPLET"])

# SWAPTIONS
for e in swaption_expiries:
    for t in swaption_tenors:
        rows.append([INDEX, float(e), float(t),
                     float(swaption_atm_normal(e,t)),
                     float(beta_fixed),
                     float(nu_surface(e,t,"SWAPTION")),
                     float(rho_surface(e,t,"SWAPTION")),
                     "SWAPTION"])

sabr_sofr_topdown5 = pd.DataFrame(rows, columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO","PRODUCT"])

m = (
    (sabr_sofr_topdown2["INDEX"] == "SOFR-1B") &
    np.isclose(sabr_sofr_topdown2["AXIS1"], 1.0) &
    np.isclose(sabr_sofr_topdown2["AXIS2"], 0.25)
)

sabr_sofr_topdown5.loc[m, "RHO"] += 0.01

sabr_sofr_topdown5


,INDEX,AXIS1,AXIS2,NORMALVOL,BETA,NU,RHO,PRODUCT
0,SOFR-1B,0.083333,0.083333,0.013558,0.600000,1.287038,-0.287472,CAPLET
1,SOFR-1B,0.083333,0.250000,0.012913,0.600000,1.257189,-0.259746,CAPLET
2,SOFR-1B,0.083333,0.500000,0.012396,0.600000,1.227503,-0.237361,CAPLET
3,SOFR-1B,0.250000,0.083333,0.012980,0.600000,1.216044,-0.278885,CAPLET
4,SOFR-1B,0.250000,0.250000,0.012362,0.600000,1.187842,-0.251160,CAPLET
5,SOFR-1B,0.250000,0.500000,0.011868,0.600000,1.159793,-0.228775,CAPLET
6,SOFR-1B,0.500000,0.083333,0.012174,0.600000,1.118045,-0.267946,CAPLET
7,SOFR-1B,0.500000,0.250000,0.011595,0.600000,1.092116,-0.240220,CAPLET
8,SOFR-1B,0.500000,0.500000,0.011131,0.600000,1.066327,-0.217836,CAPLET
9,SOFR-1B,1.000000,0.083333,0.010759,0.600000,0.949137,-0.250685,CAPLET


In [39]:
data_objs5 = []
for idx_name, sub in sabr_sofr_topdown5.groupby("INDEX"):
    for param in ["NORMALVOL","BETA","NU","RHO"]:
        pivot = (
            sub.pivot(index="AXIS1", columns="AXIS2", values=param)
               .sort_index(axis=0).sort_index(axis=1)
        )
        data_objs5.append(Data2D.createDataObject(param.lower(), idx_name, pivot))

In [40]:
value_date = "2025-09-24"

topdown_build_methods = []
for product in ("CAPLET","SWAPTION"):
    for param in ("NORMALVOL","BETA","NU","RHO"):
        topdown_build_methods.append({
            "TARGET":           "SOFR-1B",
            "VALUES":           param,
            "AXIS1":            "AXIS1",
            "AXIS2":            "AXIS2",
            "INTERPOLATION":    "LINEAR",
            "SHIFT":            0.0,
            "VOL_DECAY_SPEED":  0.2,
            "PRODUCT":          product
        })

dc5 = DataCollection(data_objs5)

td_sabr6 = SabrModel.from_curve(
    valueDate=value_date,
    dataCollection=dc5,
    buildMethodCollection=topdown_build_methods,
    ycModel=yc
)

print(td_sabr6.components.keys())

dict_keys(['SOFR-1B-NORMALVOL-CAPLET', 'SOFR-1B-BETA-CAPLET', 'SOFR-1B-NU-CAPLET', 'SOFR-1B-RHO-CAPLET', 'SOFR-1B-NORMALVOL-SWAPTION', 'SOFR-1B-BETA-SWAPTION', 'SOFR-1B-NU-SWAPTION', 'SOFR-1B-RHO-SWAPTION'])


In [41]:
td_cap = ProductOvernightCapFloorlet(
    effectiveDate="2026-07-01",
    termOrEnd    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="CAP",
    strike       =0.018,
    notional     =1000000,
    longOrShort  ="LONG"
)

vp = {"SABR_METHOD": "top-down", "FUNDING INDEX": "SOFR-1B"}
ve = ValuationEngineRegistry().new_valuation_engine(td_sabr6, vp, td_cap)

ve.calculateValue()

f = ve.value[1]

In [42]:
(f-a)/0.01

np.float64(-62.70577026953106)